In [52]:
#env
import os
import requests
from dotenv import load_dotenv
load_dotenv()
#Langchain
from langchain_openai import AzureOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.document_loaders import WebBaseLoader

### LLM

In [53]:
llm = AzureOpenAI(
  api_key=os.getenv("AZURE_OPENAI_KEY"),  
  api_version="2023-12-01-preview",
  azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),  # Add a comma here
  deployment_name="text-davinci-003",
  model_name="text-davinci-003",
  temperature=0,
  max_tokens=1024
)

In [54]:
print(llm)

AzureOpenAI
Params: {'deployment_name': 'text-davinci-003', 'model_name': 'text-davinci-003', 'temperature': 0.0, 'top_p': 1, 'frequency_penalty': 0, 'presence_penalty': 0, 'n': 1, 'logit_bias': {}, 'max_tokens': 1024}


In [55]:
llm.invoke("how can langsmith help with testing?")

'\n\nLangsmith can help with testing by providing automated testing tools that can be used to test the accuracy of translations. These tools can be used to compare the original source language to the translated language to ensure that the translation is accurate and complete. Additionally, Langsmith can provide feedback on the quality of the translation, allowing testers to identify any errors or inconsistencies in the translation.'

### prompt template


In [56]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are world class technical documentation writer."),
    ("user", "{input}")
])

### simple LLM chain

In [57]:
chain = prompt | llm 

In [58]:
chain.invoke({"input": "how can langsmith help with testing?"})

'\n\nLangsmith can help with testing by providing automated testing tools that can be used to quickly and accurately test the accuracy of your technical documentation. The tools can be used to check for typos, grammar errors, and other inconsistencies in your documentation. Additionally, Langsmith can provide feedback on the readability and clarity of your documentation, helping you to ensure that your technical documentation is as clear and concise as possible.'

### output parser

In [59]:
output_parser = StrOutputParser()

In [60]:
chain = prompt | llm | output_parser

In [61]:
chain.invoke({"input": "how can langsmith help with testing?"})

'\n\nLangsmith can help with testing by providing automated testing tools that can be used to quickly and accurately test the accuracy of your technical documentation. The tools can be used to check for typos, grammar errors, and other inconsistencies in your documentation. Additionally, Langsmith can provide feedback on the readability and clarity of your documentation, helping you to ensure that your technical documentation is as clear and concise as possible.'

### Retrieval Chain


In [62]:
! pip install -q beautifulsoup4
! pip install -qU certifi



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [63]:
loader = WebBaseLoader("https://docs.smith.langchain.com/overview")
loader.requests_kwargs = {'verify': False}


In [64]:

docs = loader.load()

c:\Users\smaddinieni2\OneDrive - DXC Production\Desktop\Projects\Notebooks\myenv\Lib\site-packages\urllib3\connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'docs.smith.langchain.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [65]:
from langchain_openai import AzureOpenAIEmbeddings

embeddings = AzureOpenAIEmbeddings(api_key=os.getenv("AZURE_OPENAI_KEY"),model=os.getenv("AZURE_OPENAI_ADA_EMBEDDING_MODEL_NAME"),deployment=os.getenv("AZURE_OPENAI_ADA_EMBEDDING_DEPLOYMENT_NAME"))

In [66]:
print(embeddings)

client=<openai.resources.embeddings.Embeddings object at 0x0000012D842988D0> async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x0000012D842958D0> model='text-embedding-ada-002' dimensions=None deployment='text-embedding-ada-002-v2' openai_api_version='2023-05-15' openai_api_base=None openai_api_type='azure' openai_proxy='' embedding_ctx_length=8191 openai_api_key='001aee09f12a4de28089d6c2d73886b5' openai_organization=None allowed_special=set() disallowed_special='all' chunk_size=16 max_retries=2 request_timeout=None headers=None tiktoken_enabled=True tiktoken_model_name=None show_progress_bar=False model_kwargs={} skip_empty=False default_headers=None default_query=None retry_min_seconds=4 retry_max_seconds=20 http_client=None azure_endpoint='https://test-aipractice-openai.openai.azure.com/' azure_ad_token=None azure_ad_token_provider=None validate_base_url=True


In [67]:
! pip install -q faiss-cpu


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [68]:
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

In [69]:
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

In [70]:
from langchain_core.documents import Document

document_chain.invoke({
    "input": "how can langsmith help with testing?",
    "context": [Document(page_content="langsmith can let you visualize test results")]
})

'\n\nLangsmith can help with testing by allowing you to visualize the test results. This can help you identify any issues or patterns in the results more quickly and easily.'

In [71]:
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [72]:
response = retrieval_chain.invoke({"input": "how can langsmith help with testing?"})
print(response["answer"])



LangSmith can help with testing by making it easy to curate datasets and run chains over them to visualize the outputs. It also provides evaluators to guide the eye to examples that should be looked at, and annotation queues to allow for manual review and annotation of runs.


### Conversation Retrieval Chain

In [ ]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

# First we need a prompt that we can pass into an LLM to generate this search query

prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Given the above conversation, generate a search query to look up in order to get information relevant to the conversation")
])
retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

In [ ]:
from langchain_core.messages import HumanMessage, AIMessage

chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
retriever_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

In [ ]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the user's questions based on the below context:\n\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
])
document_chain = create_stuff_documents_chain(llm, prompt)

retrieval_chain = create_retrieval_chain(retriever_chain, document_chain)

In [ ]:
chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
retrieval_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

### Agent